In [513]:
from Vocabscraper import VocabularyScraper
import os
import pickle
import pandas
from copy import deepcopy
from collections import Counter
import re
import nltk
from  nltk.tokenize import word_tokenize as tokenizer
import itertools
import string
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from nltk import ngrams


In [420]:
def load_data():
    
    ''' Helper function to load dataset '''
    data=None
    FILE_PATH= "data\words.pkl"
    if not os.path.exists(FILE_PATH):
        os.makedirs("data")
        cursor = VocabularyScraper(None).db_executer
        cursor.execute("SELECT term,definition FROM terms")
        data = cursor.fetchall()
        with open(FILE_PATH,"wb") as file:
            pickle.dump(data,file)
    else:
        with open(FILE_PATH,"rb") as file:
            data = pickle.load(FILE_PATH)
    return data



In [421]:
def look_up_word(term):
    for word in data_copy:
        if word[1]==term:
            print(word)

In [49]:
def find_optimum_length(data,percentile):
    pass
    
    

In [35]:
with open("data/words.pkl","wb") as file:
    pickle.dump(words,file)

(178200, '', 'giddy; frivolous; feather-headed &$ any thin, as on a board or a razor &$ the thin, new growth around the edge of a shell, of an oyster &$ having a feather-edge; also, having one edge thinner than the other, as a board; -- in the United States, said only of stuff one edge of which is made as thin as practicable &$ feverfew', None)


In [455]:
def ex_remove_red_chars(*examples):
    result=[]
    for example in examples:
        if example is not None:
            punctuation=re.sub("[#$&]*","",string.punctuation)
            punctuation = [char for char in punctuation]
            del_patterns = ["&[\w]+&",'”',"^[\s]","“","adj\n","v\n","n\n","adv\n"]
            del_patterns.extend(punctuation)
            for pattern in del_patterns:
                example = example.replace(pattern,"")
            example = re.split(r"[&#$]*;*",example) 
            
            result.append(example)
        else:
            result.append("")
    return tuple(result)


In [ ]:
tag_transform={"adj\\n":"&JJ&","v\\n":""}

In [147]:
test

(3548,
 'adorn',
 'furnish with power or authority; of kings or emperors &$ To make more beautiful and attractive; to decorate &$ make more attractive by adding ornament, colour, etc &$ be beautiful to look at &$ To adorn is to dress something up by decorating it. You might adorn your poncho with fringe or your poodle`s dog collar with rhinestones. &$ Adorn shares some Latin roots with words like ornament and ornate. So it makes sense that some people adorn their Christmas trees with tinsel and lights. Others adorn their eyelids with glitter. In any case, if you want to adorn yourself with all kinds of fabulous baubles, it might be wise to follow Coco Chanel`s advice: take at least one accessory off before you walk out the door. &$ v\nmake more attractive by adding ornament, colour, etc. &$ v\nbe beautiful to look at &$ v\nfurnish with power or authority; of kings or emperors',
 'None &$ &VBD& “Flowers adorned the tables everywhere”')

In [180]:
help(nltk.pos_tag)

Help on function pos_tag in module nltk.tag:

pos_tag(tokens, tagset=None, lang='eng')
    Use NLTK's currently recommended part of speech tagger to
    tag the given list of tokens.
    
        >>> from nltk.tag import pos_tag
        >>> from nltk.tokenize import word_tokenize
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."))
        [('John', 'NNP'), ("'s", 'POS'), ('big', 'JJ'), ('idea', 'NN'), ('is', 'VBZ'),
        ("n't", 'RB'), ('all', 'PDT'), ('that', 'DT'), ('bad', 'JJ'), ('.', '.')]
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."), tagset='universal')
        [('John', 'NOUN'), ("'s", 'PRT'), ('big', 'ADJ'), ('idea', 'NOUN'), ('is', 'VERB'),
        ("n't", 'ADV'), ('all', 'DET'), ('that', 'DET'), ('bad', 'ADJ'), ('.', '.')]
    
    NB. Use `pos_tag_sents()` for efficient tagging of more than one sentence.
    
    :param tokens: Sequence of tokens to be tagged
    :type tokens: list(str)
    :param tagset: the tagset to be u

In [392]:
def process_duplicates(data):
    terms= set()
    defs_exs=set()
   
    
    for word in data_copy:
        terms.add(word[1])
        definitions,examples = word[2],word[3]
        definitions,examples =  ex_remove_red_chars(definitions,examples)
        defs_exs.update(definitions,examples)
        
    return(terms,defs_exs)    


In [456]:
terms , defs_exs = process_duplicates(data_copy)


/Users/mac/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [458]:
counter=Counter(defs_exs)

In [459]:
all_text = list(itertools.chain(*[sent.split() for sent in list(defs_exs)]))

In [460]:
counter=Counter(all_text)

In [461]:
counter.most_common(10)


[('of', 426762),
 ('a', 420583),
 ('the', 402261),
 ('or', 263775),
 ('to', 256051),
 ('in', 173051),
 ('and', 158930),
 ('is', 84525),
 ('A', 79785),
 ('that', 78088)]

In [462]:
index=0
word_to_index=dict()
for key,indx in counter.items():
    word_to_index[key]=index
    index+=1
for word in terms:
    if word not in word_to_index:
        word_to_index[word]=index
        index+=1

In [463]:
index_to_word=dict()
for key,index in word_to_index.items():
    index_to_word[index]=key
    
    

In [427]:
look_up_word("person")

(290097, 'person', 'a human body (usually including the clothing &$ A single human being; an individual &$ A linguistic category used to distinguish between the speaker of an utterance and those to whom or about whom he is speaking. See grammatical person &$ one of three relations or conditions (that of speaking, that of being spoken to, and that of being spoken of) pertaining to a noun or a pronoun, and thence also to the verb of which it may be the subject &$ a living, self-conscious being, as distinct from an animal or a thing; a moral agent; a human being; a man, woman, or child &$ Someone who likes or has an affinity for (a specified thing &$ a character or part, as in a play; a specific kind or manifestation of individual character, whether in real life, or in literary or dramatic representation; an assumed character &$ the bodily form of a human being; body; outward appearance; as, of comely person &$ a grammatical category used in the classification of pronouns, possessive dete

In [504]:
sentences=[]
for example in defs_exs:
    sentence=[]
    for word in example.split():
        sentence.append(word_to_index[word])
    sentences.append(sentence) 
        

In [568]:
encoded=np.array(sentences)

In [569]:
encoded

array([[], [0, 1, 2, 3, 4, 5, 6, 7],
       [8, 9, 10, 11, 12, 13, 14, 15, 13, 16], ...,
       [780, 369, 15, 746, 697, 15, 2740, 14767, 13, 1109, 185059, 63, 291, 185060, 793, 15, 13, 688, 15, 13, 27535, 11714],
       [369, 15, 746, 7300, 10300, 2308, 28, 9479, 13, 79168, 15, 9645, 63, 28, 190, 13, 3234, 7545, 15, 6453, 63, 3912, 83, 8520],
       [8, 682, 6, 7891, 15, 30, 4376]], dtype=object)

In [524]:
def skipgrams(sequence, n, k):
    for ngram in ngrams(sequence, n + k, pad_right=True):
        head = ngram[:1]
        tail = ngram[1:]
        for skip_tail in itertools.combinations(tail, n - 1):
            if skip_tail[-1] is None:
                continue
            yield head + skip_tail

In [535]:
list(skipgrams(encoded[1],6,3))

[(0, 1, 2, 3, 4, 5),
 (0, 1, 2, 3, 4, 6),
 (0, 1, 2, 3, 4, 7),
 (0, 1, 2, 3, 5, 6),
 (0, 1, 2, 3, 5, 7),
 (0, 1, 2, 3, 6, 7),
 (0, 1, 2, 4, 5, 6),
 (0, 1, 2, 4, 5, 7),
 (0, 1, 2, 4, 6, 7),
 (0, 1, 2, 5, 6, 7),
 (0, 1, 3, 4, 5, 6),
 (0, 1, 3, 4, 5, 7),
 (0, 1, 3, 4, 6, 7),
 (0, 1, 3, 5, 6, 7),
 (0, 1, 4, 5, 6, 7),
 (0, 2, 3, 4, 5, 6),
 (0, 2, 3, 4, 5, 7),
 (0, 2, 3, 4, 6, 7),
 (0, 2, 3, 5, 6, 7),
 (0, 2, 4, 5, 6, 7),
 (0, 3, 4, 5, 6, 7),
 (1, 2, 3, 4, 5, 6),
 (1, 2, 3, 4, 5, 7),
 (1, 2, 3, 4, 6, 7),
 (1, 2, 3, 5, 6, 7),
 (1, 2, 4, 5, 6, 7),
 (1, 3, 4, 5, 6, 7),
 (2, 3, 4, 5, 6, 7)]

In [514]:
encoded

array([[], [0, 1, 2, 3, 4, 5, 6, 7],
       [8, 9, 10, 11, 12, 13, 14, 15, 13, 16], ...,
       [780, 369, 15, 746, 697, 15, 2740, 14767, 13, 1109, 185059, 63, 291, 185060, 793, 15, 13, 688, 15, 13, 27535, 11714],
       [369, 15, 746, 7300, 10300, 2308, 28, 9479, 13, 79168, 15, 9645, 63, 28, 190, 13, 3234, 7545, 15, 6453, 63, 3912, 83, 8520],
       [8, 682, 6, 7891, 15, 30, 4376]], dtype=object)

In [559]:
def get_batches(): 
    batches = []
    window_size=3
    
    vocab_len = len(word_to_index)
    for value in encoded[:10]:
        length = len(value)
        batch=None
        
        for index,word in enumerate(value):
            if index-window_size<0:
                range1= value[0:index]
                range2= value[index+1:index+window_size+(window_size-len(range1))+1]
                batch=range1+range2
                print(batch)
                
            if index >= length - window_size :
                range1= value[index-window_size-window_size-len(range2):index+1]
                range2=value[index+1:length]
                batch=range1+range2
            else:
                batch=value[index-window_size:index]+value[index+1:index+window_size+1]
                
                
            batches.append(batch)    
              
            
    
        
    
    
    




In [567]:
encoded[8]

[95, 96, 97, 30, 98]

In [496]:
stats = [len(ex) for ex in defs_exs]

In [497]:
df= pandas.DataFrame(stats)

In [498]:
df.describe(percentiles=[.15, .90, .99])

,0
count,706896.000000
mean,72.132277
std,73.427707
min,0.000000
15%,23.000000
50%,50.000000
90%,142.000000
99%,415.000000
max,2466.000000
